In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model and tokenizer
model_name = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU if available
model = model.to(device)

# Correct language codes for NLLB-200
# NOTE: NLLB-200 uses 'khk_Cyrl' for Mongolian (Halh Mongolian in Cyrillic script)
LANG_CODES = {
    "english": "eng_Latn",
    "mongolian": "khk_Cyrl",  # Fixed: was "mon_Cyrl", should be "khk_Cyrl"
}

def translate_text(text, source_lang, target_lang):
    """
    Translate text using NLLB-200 model
    
    Args:
        text (str): Text to translate
        source_lang (str): Source language code (e.g., "eng_Latn", "khk_Cyrl")
        target_lang (str): Target language code (e.g., "eng_Latn", "khk_Cyrl")
    
    Returns:
        str: Translated text
    """
    # Tokenize input text and move to GPU
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get the target language token ID for forced decoding
    target_lang_id = tokenizer.convert_tokens_to_ids(target_lang)
    
    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=target_lang_id,
            max_length=512,
            num_beams=5,
            early_stopping=True,
            do_sample=False,
            temperature=1.0,
        )
    
    # Decode the output
    translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    return translated_text

def english_to_mongolian(text):
    """Translate English text to Mongolian"""
    return translate_text(text, LANG_CODES["english"], LANG_CODES["mongolian"])

def mongolian_to_english(text):
    """Translate Mongolian text to English"""
    return translate_text(text, LANG_CODES["mongolian"], LANG_CODES["english"])


Using device: cuda


model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

In [6]:
import time

# Test English to Mongolian
english_text = """Mongolia[b] is a landlocked country in East Asia, bordered by Russia to the north and China to the south and southeast. It covers an area of 1,564,116 square kilometres (603,909 square miles), with a population of 3.5 million, making it the world's most sparsely populated sovereign state. Mongolia is the world's largest landlocked country that does not border an inland sea, and much of its area is covered by grassy steppe, with mountains to the north and west and the Gobi Desert to the south. Ulaanbaatar, the capital and largest city, is home to roughly half of the country's population.

The territory of modern-day Mongolia has been ruled by various nomadic empires, including the Xiongnu, the Xianbei, the Rouran, the First Turkic Khaganate, the Second Turkic Khaganate, the Uyghur Khaganate and others. In 1206, Genghis Khan founded the Mongol Empire, which became the largest contiguous land empire in history. His grandson Kublai Khan conquered China proper and established the Yuan dynasty. After the collapse of the Yuan, the Mongols retreated to Mongolia and resumed their earlier pattern of factional conflict, except during the era of Dayan Khan and Tumen Zasagt Khan.

In the 16th century, Tibetan Buddhism spread to Mongolia, being further led by the Manchu-founded Qing dynasty, which absorbed the country in the 17th century. By the early 20th century, almost one-third of the adult male population were Buddhist monks.[13][14] After the collapse of the Qing dynasty in 1911, Mongolia declared independence, and achieved actual independence from the Republic of China in 1921. Shortly thereafter, the country became a satellite state of the Soviet Union. In 1924, the Mongolian People's Republic was founded as a socialist state.[15] After the anti-communist revolutions of 1989, Mongolia conducted its own peaceful democratic revolution in early 1990. This led to a multi-party system, a new constitution of 1992, and transition to a market economy.

Approximately 30% of the population is nomadic or semi-nomadic; horse culture remains integral. Buddhism is the majority religion (51.7%), with the nonreligious being the second-largest group (40.6%). Islam is the third-largest religious identification (3.2%), concentrated among ethnic Kazakhs. The vast majority of citizens are ethnic Mongols, with roughly 5% of the population being Kazakhs, Tuvans, and other ethnic minorities, who are especially concentrated in the western regions. Mongolia is a member of the United Nations, Asia Cooperation Dialogue, G77, Asian Infrastructure Investment Bank, Non-Aligned Movement and a NATO global partner. Mongolia joined the World Trade Organization in 1997 and seeks to expand its participation in regional economic and trade groups."""


en_texts = english_text.split('.')

start_time = time.time()
mongolian_translation = [english_to_mongolian(en_text) for en_text in en_texts]
end_time = time.time()



In [11]:
print(f"English: {english_text}")
print(f'Mongolian: {" ".join(mongolian_translation)}')
print(f"Time taken (English to Mongolian): {end_time - start_time:.4f} seconds")


English: Mongolia[b] is a landlocked country in East Asia, bordered by Russia to the north and China to the south and southeast. It covers an area of 1,564,116 square kilometres (603,909 square miles), with a population of 3.5 million, making it the world's most sparsely populated sovereign state. Mongolia is the world's largest landlocked country that does not border an inland sea, and much of its area is covered by grassy steppe, with mountains to the north and west and the Gobi Desert to the south. Ulaanbaatar, the capital and largest city, is home to roughly half of the country's population.

The territory of modern-day Mongolia has been ruled by various nomadic empires, including the Xiongnu, the Xianbei, the Rouran, the First Turkic Khaganate, the Second Turkic Khaganate, the Uyghur Khaganate and others. In 1206, Genghis Khan founded the Mongol Empire, which became the largest contiguous land empire in history. His grandson Kublai Khan conquered China proper and established the Y

In [12]:
# Test Mongolian to English
mongolian_text = """  Монгол Улс[10] нь дорнод болон төв Азид орших бүрэн эрхт улс. Хойгуураа Оросын Холбооны Улс, өмнө талаараа БНХАУ-тай хиллэдэг, далайд гарцгүй улс юм. Улсын нийслэл нь Улаанбаатар бөгөөд улс доторх цорын ганц их хүн амтай хот.

Монгол Улс нь 1,564,116 км2 талбай нутаг дэвсгэртэй, 2022 оны байдлаар 3.398 сая гаруй хүн амтай байна. Нийт хүн амын 95 хувийг монгол үндэстэн, 4 гаруй хувийг Казах үндэстэн эзэлдэг.

Өнөөгийн Монгол Улсын нутаг дэвсгэр дээр хүн анх суурьшсан цагаас эхлэн нүүдэлчин соёл иргэншил эзэгнэн, Монголын Хүннү, Сяньби, Нирун, Түрэг Уйгур, Кидан улс байгуулагдан оршин тогтнож байв. 1206 онд Чингис хаан Их Монгол Улсыг байгуулан Евроазийг эзэлсэн эзэнт гүрнийг үүсгэн байгуулжээ. Түүний ач хүү Хубилай хаан Хятадын эх газрыг бүхэлд нь эзэлж, Юань улсыг (1271-1368) байгуулж улсын нийслэлийг Дайду буюу одоогийн Бээжинд шилжүүлсэн байна. Юань улс унасны дараа төрийн нэгдмэл захиргаа суларч монголчууд бие даасан шинжтэй хэд хэдэн хэсэгт хуваагдан орших болсон.

17-18-р зуунд Зүрчидийн Манж Чин улсын мэдэлд орсон ч 1911 онд тусгаар тогтнолоо сэргээн тогтоосон. 20-р зуунд Монгол улс нь олон улсад тусгаар тогтнолоо бүрэн олж авсны дараа нийгэм, эдийн засаг, төр захиргааны хувьд Зөвлөлт Холбоот Улс тэргүүтэй социалист орнуудтай хамтын ажиллагаа бүхий холбоотон улс байв. 1924 онд түүхэндээ анх удаа Үндсэн хуультай болж, Бүгд Найрамдах Ардын засаглалыг тунхаглаж, социалист орон болсон.

1990 оны Ардчилсан хувьсгалаар Монгол Улс социализмаас ардчилсан, чөлөөт зах зээлийн эдийн засагт нийгэмд шилжин орсон.

Монгол Улс нь хөгжиж буй орон юм. Дотоодын нийт бүтээгдэхүүний хэмжээгээр (26.8 тэрбум ам.доллар) дэлхийд 111-т жагсаж, нэг хүнд ноогдох хэмжээ 11,882 $ байна. Хөдөө аж ахуй, нүүдлийн мал аж ахуйн уламжлалтай. Зэс, нүүрс экспортолдог. Мөнгөний нэгж — төгрөг.

Монгол Улс нь НҮБ-ын 191 улс болон Гэгээн Ширээт Улс, Палестин, Европын Холбоотой дипломат харилцаа тогтоосон бөгөөд Нэгдсэн Үндэстний Байгууллага, Дэлхийн Худалдааны Байгууллага, Олон Улсын Валютын Сан, Европын Аюулгүй Байдал Хамтын Ажиллагааны Байгууллагын гишүүн улс юм. """

mon_texts = mongolian_text.split('.')

start_time = time.time()

english_translation = [mongolian_to_english(mon_text) for mon_text in mon_texts]

end_time = time.time()


print(f"\nMongolian: {mongolian_text}")
print(f'English: {" ".join(english_translation)}')
print(f"Time taken (Mongolian to English): {end_time - start_time:.4f} seconds")


Mongolian:   Монгол Улс[10] нь дорнод болон төв Азид орших бүрэн эрхт улс. Хойгуураа Оросын Холбооны Улс, өмнө талаараа БНХАУ-тай хиллэдэг, далайд гарцгүй улс юм. Улсын нийслэл нь Улаанбаатар бөгөөд улс доторх цорын ганц их хүн амтай хот.

Монгол Улс нь 1,564,116 км2 талбай нутаг дэвсгэртэй, 2022 оны байдлаар 3.398 сая гаруй хүн амтай байна. Нийт хүн амын 95 хувийг монгол үндэстэн, 4 гаруй хувийг Казах үндэстэн эзэлдэг.

Өнөөгийн Монгол Улсын нутаг дэвсгэр дээр хүн анх суурьшсан цагаас эхлэн нүүдэлчин соёл иргэншил эзэгнэн, Монголын Хүннү, Сяньби, Нирун, Түрэг Уйгур, Кидан улс байгуулагдан оршин тогтнож байв. 1206 онд Чингис хаан Их Монгол Улсыг байгуулан Евроазийг эзэлсэн эзэнт гүрнийг үүсгэн байгуулжээ. Түүний ач хүү Хубилай хаан Хятадын эх газрыг бүхэлд нь эзэлж, Юань улсыг (1271-1368) байгуулж улсын нийслэлийг Дайду буюу одоогийн Бээжинд шилжүүлсэн байна. Юань улс унасны дараа төрийн нэгдмэл захиргаа суларч монголчууд бие даасан шинжтэй хэд хэдэн хэсэгт хуваагдан орших болсон.

17

In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Initialize Flask app
app = Flask(__name__)

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model & tokenizer
model_name = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Language codes
LANG_CODES = {
    "english": "eng_Latn",
    "mongolian": "khk_Cyrl",
}

# Translation logic
def translate_text(text, source_lang, target_lang):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    target_lang_id = tokenizer.convert_tokens_to_ids(target_lang)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=target_lang_id,
            max_length=512,
            num_beams=5,
            early_stopping=True,
            do_sample=False,
            temperature=1.0,
        )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Endpoint: English → Mongolian
@app.route('/en2mon', methods=['POST'])
def en_to_mon():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "Missing 'text' field"}), 400

    translated = translate_text(data['text'], LANG_CODES["english"], LANG_CODES["mongolian"])
    return jsonify({"translation": translated})

# Endpoint: Mongolian → English
@app.route('/mon2en', methods=['POST'])
def mon_to_en():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "Missing 'text' field"}), 400

    translated = translate_text(data['text'], LANG_CODES["mongolian"], LANG_CODES["english"])
    return jsonify({"translation": translated})

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Using device: cuda


model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.102.248:5000
Press CTRL+C to quit
